## Packages import

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
!pip install pytesseract

import os
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import json

# Optional OCR tools
import pytesseract
from PIL import Image
# or use EasyOCR:
# import easyocr

In [3]:
# PyTorch
import torch

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'使用 GPU：{torch.cuda.get_device_name(0)}')
else:
    device = torch.device('cpu')
    print('使用 CPU')

使用 GPU：NVIDIA L4


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
os.chdir("/content/drive/MyDrive/Database")

## Pre Processing

In [39]:
def load_and_merge(subject: str) -> pd.DataFrame:
    """
    Load question bank and chapter list CSVs for a given subject from Database/{subject}_Database and merge on chapter ID.

    Args:
        subject: 'math' or 'social'
    Returns:
        DataFrame with columns: question_text, chapter_name, subject
    """
    base_path = f"{subject}_Database"
    qbank_path = f"{base_path}/{subject}_question_bank.csv"
    chap_path = f"{base_path}/{subject}_chapter_list.csv"

    qdf = pd.read_csv(qbank_path)
    cdf = pd.read_csv(chap_path)
    qdf.columns = qdf.columns.str.strip().str.lower().str.replace(' ', '_')
    cdf.columns = cdf.columns.str.strip().str.lower().str.replace(' ', '_')

    print("Question Bank Columns:", qdf.columns.tolist())
    print("Chapter List Columns:", cdf.columns.tolist())


    df = qdf.merge(cdf, on='section_name', how='left')
    df['subject'] = subject
    return df

# Load and combine data from correct paths
# df = pd.concat([load_and_merge('math'), load_and_merge('social'), load_and_merge('chinese'), load_and_merge('science'), load_and_merge('english')], ignore_index=True)
df = pd.concat([load_and_merge('math'), load_and_merge('social'), load_and_merge('science')], ignore_index=True)
# df = pd.concat([load_and_merge('chinese')], ignore_index=True)

df.count()
df.head()


Question Bank Columns: ['subject', 'ques_no', 'chapter_name', 'section_name', 'ques_detl']
Chapter List Columns: ['year_grade', 'book', 'chapter_num', 'chapter_name', 'section_num', 'section_name']
Question Bank Columns: ['subject', 'ques_no', 'chapter_name', 'section_name', 'ques_detl']
Chapter List Columns: ['year_grade', 'book', 'chapter_num', 'chapter_name', 'section_num', 'section_name']
Question Bank Columns: ['subject', 'ques_no', 'chapter_name', 'section_name', 'ques_detl']
Chapter List Columns: ['year_grade', 'book', 'chapter_num', 'chapter_name', 'section_num', 'section_name']


,subject,ques_no,chapter_name_x,section_name,ques_detl,year_grade,book,chapter_num,chapter_name_y,section_num
0,math,2MA0810001,整數的運算,負數與數線,以中午12時為基準，下午3時記作＋3，那麼上午10時可以記作多少？(A)＋10(B)－10(...,8.0,1.0,1.0,整數的運算,1.0
1,math,2MA0810002,整數的運算,負數與數線,若數學科成績以30分為基準，得40分記作＋10，那麼考25分可記作多少？ (A)－25 (B...,8.0,1.0,1.0,整數的運算,1.0
2,math,2MA0810003,整數的運算,負數與數線,如果以正午12時為基準，當日下午4時用＋4表示，則當日上午8時應記為多少？ (A)－8 (B...,8.0,1.0,1.0,整數的運算,1.0
3,math,2MA0810004,整數的運算,負數與數線,如果以中午12時為基準，當日下午2時記為＋8，則當日上午6時應記為多少？(A)－24(B)－...,8.0,1.0,1.0,整數的運算,1.0
4,math,2MA0810005,整數的運算,負數與數線,如果＋40°表示北緯40°，則南緯30°該如何表示？ (A)30° (B)－30° (C)...,8.0,1.0,1.0,整數的運算,1.0


In [40]:
df = df[['subject','chapter_name_x','section_name','ques_detl']]
df.head()

,subject,chapter_name_x,section_name,ques_detl
0,math,整數的運算,負數與數線,以中午12時為基準，下午3時記作＋3，那麼上午10時可以記作多少？(A)＋10(B)－10(...
1,math,整數的運算,負數與數線,若數學科成績以30分為基準，得40分記作＋10，那麼考25分可記作多少？ (A)－25 (B...
2,math,整數的運算,負數與數線,如果以正午12時為基準，當日下午4時用＋4表示，則當日上午8時應記為多少？ (A)－8 (B...
3,math,整數的運算,負數與數線,如果以中午12時為基準，當日下午2時記為＋8，則當日上午6時應記為多少？(A)－24(B)－...
4,math,整數的運算,負數與數線,如果＋40°表示北緯40°，則南緯30°該如何表示？ (A)30° (B)－30° (C)...


In [41]:
df.count()

,0
subject,40341
chapter_name_x,40341
section_name,40341
ques_detl,40341


In [42]:
len(df)

40341

In [43]:
# 2. Label Encoding chapter name
# Combine subject and chapter_name as classification label

df['label_str'] = df['subject'] + '::' + df['chapter_name_x']
label2id = {lab: i for i, lab in enumerate(sorted(df['label_str'].unique()))}
id2label = {i: lab for lab, i in label2id.items()}
df['label'] = df['label_str'].map(label2id)

In [44]:
# 2. Label Encoding section name

df['label_str'] = df['subject'] + '::' + df['chapter_name_x'] + '::' + df['section_name']
label2id = {lab: i for i, lab in enumerate(sorted(df['label_str'].unique()))}
id2label = {i: lab for lab, i in label2id.items()}  # ← 這一行加上
df['label'] = df['label_str'].map(label2id)

In [45]:
label_counts = df['label'].value_counts()
valid_labels = label_counts[label_counts >= 2].index
df = df[df['label'].isin(valid_labels)]
print(len(df))

40339


In [46]:
# 3. Train/Test Split
train_df, test_df = train_test_split(df, test_size=0.1, stratify=df['label'], random_state=42)

# 4. Tokenization and Dataset Definition
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

class QDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer(text,
                                 padding='max_length',
                                 truncation=True,
                                 max_length=self.max_len,
                                 return_tensors='pt')
        item = {k: v.squeeze() for k, v in inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = QDataset(train_df['ques_detl'].tolist(), train_df['label'].tolist(), tokenizer)
test_dataset = QDataset(test_df['ques_detl'].tolist(), test_df['label'].tolist(), tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [47]:
# 5. Model Initialization
n_labels = len(label2id)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=n_labels)

# 6. Training Arguments and Trainer Setup
training_args = TrainingArguments(
    output_dir='./outputs',
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_steps=50,
    load_best_model_at_end=True,
    learning_rate=6e-5, # default 5e-5
    metric_for_best_model='accuracy'
)

# Define metric computation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = (preds == labels).mean()
    return {'accuracy': acc}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-47-264d7e1a425d>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [48]:
# 7. Train and Evaluate
def train():
    trainer.train()
    metrics = trainer.evaluate()
    print(f"Test Accuracy: {metrics['eval_accuracy']:.4f}")

# 8. Save Model and Label Mappings
def save_model():
    os.makedirs('./saved_model', exist_ok=True)
    model.save_pretrained('./saved_model')
    tokenizer.save_pretrained('./saved_model')
    with open('./saved_model/label2id.json', 'w') as f:
        json.dump(label2id, f)
    with open('./saved_model/id2label.json', 'w') as f:
        json.dump(id2label, f)
    torch.save(model.state_dict(), './saved_model/model.pt')

In [49]:
# 9. Inference Pipeline
def ocr_to_text(image_path: str) -> str:
    # Using pytesseract
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img, lang='eng')
    return text.strip()
    # Or using EasyOCR:
    # reader = easyocr.Reader(['en'])
    # result = reader.readtext(image_path, detail=0)
    # return ' '.join(result)


def load_classifier(model_dir: str = './saved_model'):
    tokenizer = BertTokenizerFast.from_pretrained(model_dir)
    model = BertForSequenceClassification.from_pretrained(model_dir)
    model.eval()
    return tokenizer, model


def predict_text(text: str, tokenizer, model):
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_id = logits.argmax(dim=-1).item()
    label = id2label[pred_id]
    subject, chapter, section = label.split('::')
    return subject, chapter, section


def test_image(image_path: str):
    print("Processing:", image_path)
    text = ocr_to_text(image_path)
    print("OCR Result:\n", text)
    tokenizer, model = load_classifier()
    subject, chapter_name, section_name = predict_text(text, tokenizer, model)
    print(f"Predicted Subject: {subject}\nPredicted Chapter: {chapter_name}")

def test_text(text):
    tokenizer, model = load_classifier()
    subject, chapter_name, section_name = predict_text(text, tokenizer, model)
    print(f"Predicted Subject: {subject}\nPredicted Chapter: {chapter_name}")

# Usage examples
if __name__ == '__main__':
    # 1. Train and save
    train()
    save_model()

    # 2. Test with an example image
    test_image('test_math.png')

    test = "根據經濟部水利署的統計，截至2019年底，臺灣40座主要水庫中，淤積率超過30%的共有15座，例如霧社水庫淤積率達74.8%、烏山頭水庫達49.2%。顯示臺灣水庫淤積程度嚴重，影響水庫蓄水功能。下列何項策略最能有效改善上述現象"
    test_text(test)


Epoch,Training Loss,Validation Loss,Accuracy
1,5.122000,5.108679,0.043381
2,4.573600,4.693695,0.068914
3,4.347900,4.293744,0.100397
4,3.942800,3.919680,0.150719
5,3.627500,3.628955,0.201041
6,3.288900,3.391607,0.243927
7,2.946200,3.247724,0.275409
8,2.890000,3.189430,0.288795


Test Accuracy: 0.2888
Processing: test_math.png


FileNotFoundError: [Errno 2] No such file or directory: 'test_math.png'

In [52]:
# 9. Inference Pipeline
def ocr_to_text(image_path: str) -> str:
    # Using pytesseract
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img, lang='eng')
    return text.strip()
    # Or using EasyOCR:
    # reader = easyocr.Reader(['en'])
    # result = reader.readtext(image_path, detail=0)
    # return ' '.join(result)


def load_classifier(model_dir: str = './saved_model'):
    tokenizer = BertTokenizerFast.from_pretrained(model_dir)
    model = BertForSequenceClassification.from_pretrained(model_dir)
    model.eval()
    return tokenizer, model


def predict_text(text: str, tokenizer, model):
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_id = logits.argmax(dim=-1).item()
    label = id2label[pred_id]
    subject, chapter, section = label.split('::')
    return subject, chapter, section


def test_image(image_path: str):
    print("Processing:", image_path)
    text = ocr_to_text(image_path)
    print("OCR Result:\n", text)
    tokenizer, model = load_classifier()
    subject, chapter_name, section_name = predict_text(text, tokenizer, model)
    print(f"Predicted Subject: {subject}\nPredicted Chapter: {chapter_name}")

def test_text(text):
    tokenizer, model = load_classifier()
    subject, chapter_name, section_name = predict_text(text, tokenizer, model)
    print(f"Predicted Subject: {subject}\nPredicted Chapter: {chapter_name}\nPredicted Section: {section_name}")

test = "根據經濟部水利署的統計，截至2019年底，臺灣40座主要水庫中，淤積率超過30%的共有15座，例如霧社水庫淤積率達74.8%、烏山頭水庫達49.2%。顯示臺灣水庫淤積程度嚴重，影響水庫蓄水功能。下列何項策略最能有效改善上述現象"
test_text(test)

Predicted Subject: social
Predicted Chapter: 地理：基本概念與臺灣
Predicted Section: 臺灣的水文臺灣的水資源夠用嗎？


In [ ]:
from torch.nn.functional import softmax

outputs = model(**inputs)
logits = outputs.logits         # shape: [batch_size, num_labels]
probs = softmax(logits, dim=-1) # shape: [batch_size, num_labels]


In [ ]:
output = model(x)                  # shape: [batch_size, num_classes]
probs = torch.sigmoid(output)     # 每個類別機率獨立在 0～1
preds = (probs > 0.5).float()     # 每類是否為正類別

In [ ]:
topk = torch.topk(probs, k=3, dim=-1)
topk_indices = topk.indices[0].tolist()  # 取第一筆 sample
topk_probs = topk.values[0].tolist()

for idx, prob in zip(topk_indices, topk_probs):
    print(f"{id2label[idx]}: {prob:.3f}")


## textCNN pipeline

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast
import json
from tqdm import tqdm

In [ ]:
# 3. TextCNN Model Definition
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, kernel_sizes=[3,4,5], num_filters=100):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=tokenizer.pad_token_id)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embed_dim,
                      out_channels=num_filters,
                      kernel_size=k)
            for k in kernel_sizes
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)

    def forward(self, input_ids, attention_mask=None):
        x = self.embedding(input_ids)  # [B, L, D]
        x = x.permute(0, 2, 1)        # [B, D, L]
        convs = [torch.relu(conv(x)) for conv in self.convs]  # list of [B, F, L-k+1]
        pools = [torch.max(c, dim=2)[0] for c in convs]     # list of [B, F]
        out = torch.cat(pools, dim=1)                       # [B, F*len]
        out = self.dropout(out)
        return self.fc(out)

# 4. Training and Evaluation Functions
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_cnn = TextCNN(vocab_size=len(tokenizer), embed_dim=128,
                    num_classes=len(label2id)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_cnn.parameters(), lr=1e-3)

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, total_correct = 0, 0
    for batch in tqdm(loader, desc='Train'):
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * input_ids.size(0)
        preds = outputs.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

def eval_epoch(model, loader, criterion):
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for batch in tqdm(loader, desc='Eval'):
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * input_ids.size(0)
            preds = outputs.argmax(dim=1)
            total_correct += (preds == labels).sum().item()
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

# 5. Run Training and Compare
epochs = 5
best_acc = 0
for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model_cnn, train_loader, optimizer, criterion)
    val_loss, val_acc = eval_epoch(model_cnn, test_loader, criterion)
    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model_cnn.state_dict(), 'best_textcnn.pt')

print(f"Best TextCNN Val Acc: {best_acc:.4f}")

# 6. Save Mappings for CNN
with open('tokenizer_vocab.json', 'w') as f:
    json.dump(tokenizer.vocab, f)
with open('label2id.json', 'w') as f:
    json.dump(label2id, f)